In [1]:
import argparse
import sys
import os
import time
import datetime
import ast
from collections import Counter
from pathlib import Path
import hydra

import numpy as np
import torch

# Ensure local src/ is importable
sys.path.insert(0, str(Path(os.getcwd()).parent / "src"))

from my_genQC.inference.eval_metrics import UnitaryFrobeniusNorm, UnitaryInfidelityNorm
from my_genQC.inference.evaluation_helper import get_unitaries, get_srvs
from my_genQC.inference.sampling import generate_compilation_tensors, generate_tensors, decode_tensors_to_backend
from my_genQC.pipeline.diffusion_pipeline import DiffusionPipeline
from my_genQC.platform.simulation import Simulator, CircuitBackendType
from my_genQC.platform.tokenizer.circuits_tokenizer import CircuitTokenizer
from my_genQC.utils.misc_utils import infer_torch_device, get_entanglement_bins
from my_genQC.dataset import circuits_dataset
from my_genQC.models.config_model import ConfigModel
from my_genQC.utils.config_loader import load_config, store_tensor, load_tensor

In [2]:
def load_dataset(dataset_path: Path, device: torch.device):
    config_path = os.path.join(dataset_path, "config.yaml")

    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found at {config_path}")

    cfg_data = load_config(config_path)
    target = cfg_data.get("target", "")
    if target.endswith("MixedCircuitsConfigDataset"):
        dataset_cls = circuits_dataset.MixedCircuitsConfigDataset
    else:
        dataset_cls = circuits_dataset.CircuitsConfigDataset

    dataset = dataset_cls.from_config_file(
        config_path=config_path,
        device=device,
        save_path=os.path.join(dataset_path, "dataset", "ds")
    )
    
    return dataset

ds = load_dataset("../datasets/paper_quditkit/srv_3q_dataset", device="cpu")

[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_x.safetensors` onto device: cpu.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_y.safetensors` onto device: cpu.
[INFO]: Instantiated config_dataset from given config on cpu.


In [3]:
tensor_out = load_tensor("../scripts/inference/8q_599936_samples.pt", device="cpu")

[INFO]: Loading tensor from `../scripts/inference/8q_599936_samples.pt` onto device: cpu.


In [4]:
vocabulary = {gate: idx for idx, gate in enumerate(ds.gate_pool)}
tokenizer = CircuitTokenizer(vocabulary)
simulator = Simulator(CircuitBackendType.QUDITKIT)

[WARNING]: The value 0 is reserved for background tokens, i.e. qubit time position which are not effected by gates.
[WARNING]: Automatically incrementing all vocabulary values by one ...


In [ ]:
decoded_circuits, _ = decode_tensors_to_backend(
    simulator=simulator,
    tokenizer=tokenizer,
    tensors=tensor_out,
    silent=False,
    params=None,
    n_jobs=16,
    filter_errs=False,
)